In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [42]:
import pandas as pd

df=pd.read_csv('gdrive/MyDrive/Data/GlobalLandTemperaturesByCountry.csv') #Temperature df
df_co2=pd.read_csv('gdrive/MyDrive/Data/CO2emissionbycountries.csv',encoding= 'unicode_escape') #CO2 df

In [47]:
df.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [48]:
df_co2.head(5)

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2)
0,Afghanistan,AF,93,1750,0.0,41128771.0,652230.0,0.40%,63/km²
1,Afghanistan,AF,93,1751,0.0,41128771.0,652230.0,0.40%,63/km²
2,Afghanistan,AF,93,1752,0.0,41128771.0,652230.0,0.40%,63/km²
3,Afghanistan,AF,93,1753,0.0,41128771.0,652230.0,0.40%,63/km²
4,Afghanistan,AF,93,1754,0.0,41128771.0,652230.0,0.40%,63/km²


In [49]:
#Dropping data with CO2 emission equal 0
df= df_co2[df_co2['CO2 emission (Tons)'] != 0]

In [50]:
#Preparing data by dropping duplicates to visualize in which years surveys start.
df_single=df
df_single=df_single.drop_duplicates(subset=['Country'])
print(df_single)

                 Country Code Calling Code  Year  CO2 emission (Tons)  \
199          Afghanistan   AF           93  1949              14656.0   
454              Albania   AL          355  1933               7328.0   
708              Algeria   DZ          213  1916               3664.0   
1053             Andorra   AD          376  1990             406704.0   
1284              Angola   AO          244  1950             186864.0   
...                  ...  ...          ...   ...                  ...   
58407            Vietnam   VN           84  1892             212512.0   
58787  Wallis and Futuna   WF          681  2001              14656.0   
59007              Yemen   YE          967  1950              58624.0   
59278             Zambia   ZM          260  1950            2330187.0   
59502           Zimbabwe   ZW          263  1903             113584.0   

       Population(2022)       Area % of World Density(km2)  
199          41128771.0   652230.0      0.40%       63/km²  
4

In [51]:
#Filtering country with data surveys starting from 1959
df_single = df_single[df_single['Year'] > 1959]
print(df_single)

                        Country Code Calling Code  Year  CO2 emission (Tons)  \
1053                    Andorra   AD          376  1990             406704.0   
1595                   Anguilla   AI        1-264  1990              51296.0   
1863                 Antarctica   AQ          672  1987               3664.0   
6453                     Bhutan   BT          975  1970               3664.0   
7268                   Botswana   BW          267  1972              21984.0   
11060          Christmas Island   CX           61  1970              69616.0   
12143              Cook Islands   CK          682  1969              10992.0   
27582                  Kiribati   KI          686  1961               7328.0   
27900                    Kosovo   XK          383  2008            7386624.0   
29508                   Lesotho   LS          266  1990            1472928.0   
30321             Liechtenstein   LI          423  1990             198968.0   
32199                  Maldives   MV    

In [52]:
#Left outer join exclusion on country with data surveys starting from 1959
df_final=pd.merge(df,df_single[['Country']],on=['Country'],how="outer",indicator=True)
df_final=df_final[df_final['_merge']=='left_only']

In [53]:
df_final

,Country,Code,Calling Code,Year,CO2 emission (Tons),Population(2022),Area,% of World,Density(km2),_merge
0,Afghanistan,AF,93,1949,14656.0,41128771.0,652230.0,0.40%,63/km²,left_only
1,Afghanistan,AF,93,1950,98928.0,41128771.0,652230.0,0.40%,63/km²,left_only
2,Afghanistan,AF,93,1951,190528.0,41128771.0,652230.0,0.40%,63/km²,left_only
3,Afghanistan,AF,93,1952,282128.0,41128771.0,652230.0,0.40%,63/km²,left_only
4,Afghanistan,AF,93,1953,388384.0,41128771.0,652230.0,0.40%,63/km²,left_only
...,...,...,...,...,...,...,...,...,...,...
21702,Zimbabwe,ZW,263,2016,736467042.0,16320537.0,390757.0,0.30%,42/km²,left_only
21703,Zimbabwe,ZW,263,2017,746048675.0,16320537.0,390757.0,0.30%,42/km²,left_only
21704,Zimbabwe,ZW,263,2018,757903042.0,16320537.0,390757.0,0.30%,42/km²,left_only
21705,Zimbabwe,ZW,263,2019,768852126.0,16320537.0,390757.0,0.30%,42/km²,left_only
